In [1]:
%pip install -q kagglehub
print('done')

done


In [2]:
import sentiment_dataset
X, y = sentiment_dataset.load_sentiment_tweets()

X = X.sample(n = 20000, random_state=1)
y = y.sample(n = 20000, random_state=1)
print(X)
print(y)

Dataset downloaded to: /kaggle/input/sentiment140
Loading data from /kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv...
Original dataset shape: (1600000, 6)
Shape after filtering out neutral tweets (polarity=2): (1600000, 6)
Data processing complete. Returning tweet text and binary sentiment labels.
514293     i miss nikki nu nu already  shes always there ...
142282     So I had a dream last night. I  remember a sig...
403727     @girlyghost ohh poor sickly you   (((hugs)) ho...
649503                                  it is raining again 
610789             @MissKeriBaby wish I was in LA right now 
                                 ...                        
1200592    I get the worst writer's cramp. These thank yo...
333050           @ladyinreddress the sun is all gone now... 
893698     FINALLY a break till Wed, no work OR school, b...
1145456                              I want a Blackberry... 
837125     in montrea doing some much needed ocean swimmi...
Name: te

In [3]:
import pandas as pd
import numpy as np
import re
import string

#import dataset
#data = pd.read_csv("RateMyProfessor_Sample data.csv", usecols = ["comments", "student_star"])


#preprocessing
#remove all punctuation
X = X.str.replace(r'[^\w\s]+', '', regex = True)
#make all letters lowercase
X = X.str.lower()

print(X)

514293     i miss nikki nu nu already  shes always there ...
142282     so i had a dream last night i  remember a sign...
403727     girlyghost ohh poor sickly you   hugs hope you...
649503                                  it is raining again 
610789              misskeribaby wish i was in la right now 
                                 ...                        
1200592    i get the worst writers cramp these thank you ...
333050               ladyinreddress the sun is all gone now 
893698     finally a break till wed no work or school bot...
1145456                                 i want a blackberry 
837125     in montrea doing some much needed ocean swimmi...
Name: text, Length: 20000, dtype: object


In [8]:
#feature extraction
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#list of stop words
stop_str = (['the', 'a', 'and', 'to', 'you', 'is', 'he', 'she', 'of', 'an', 'in',
            'but', 'his', 'her', 'was', 'have', 'with', 'take', 'with', 'that', 'do', 'be',
            'for', 'if', 'it', 'are', 'on', 'this', 'will', 'at', 'about', 'as', 'so', 'guy',
            'him', 'your', 'had', 'can', 'hes', 'from', 'me', 'its', 'shes', 'get', 'my', 'im'])

cv = CountVectorizer(stop_words = stop_str)
tfidf_transformer = TfidfTransformer()


#bag of words
X_bow = cv.fit_transform(X.values.astype('U'))
X_bow.shape

(20000, 32918)

In [9]:
#analysis of dataset
#class distribution, text length distribution, total unique words, most common words
import matplotlib.pyplot as plt

#print('The accuracy for Naive Bayes classifier using TF-IDF is {:.5f} on training data'.format(accuracy_score(y_pred = clf.predict(X_train), y_true = y_train)))
#pos_ct = sum(data['student_star'] > 0)
pos_ct = sum(y > 0)
#neg_ct = sum(data['student_star'] < 0)
neg_ct = sum(y == 0)

print('There are {:.0f} total records.'.format( pos_ct + neg_ct ))
print('There are {:.0f} positive records.'.format( pos_ct ))
print('There are {:.0f} negative records.'.format( neg_ct ))
print('{:.2f}% of records in the dataset are positive.\n'.format( (pos_ct / (pos_ct + neg_ct)) * 100 ))

'''
word_ct = pd.read_csv("RateMyProfessor_Sample data.csv", usecols = ["word_comment"])
plt.hist(word_ct, color='lightgreen', ec='black', bins=10)
plt.xlabel("Number of Words")
plt.ylabel("Frequency")
plt.title("Distribution of Word Count")
plt.show()
'''

print('\nThere are {:.0f} unique words.\n'.format( len(cv.get_feature_names_out()) ))

print('The 25 most common words, excluding stop words, are: ')
freqs = zip(cv.get_feature_names_out(), X_bow.sum(axis=0).tolist()[0])
# sort from largest to smallest
print( sorted(freqs, key=lambda x: -x[1]) [:25])

There are 20000 total records.
There are 10049 positive records.
There are 9951 negative records.
50.24% of records in the dataset are positive.


There are 32918 unique words.

The 25 most common words, excluding stop words, are: 
[('just', 1574), ('not', 1287), ('now', 1203), ('good', 1122), ('up', 1075), ('day', 1005), ('out', 985), ('like', 983), ('all', 961), ('go', 925), ('no', 923), ('dont', 839), ('love', 797), ('got', 772), ('too', 771), ('going', 768), ('cant', 767), ('work', 743), ('today', 725), ('back', 721), ('lol', 704), ('time', 664), ('really', 655), ('what', 652), ('one', 629)]


In [10]:
#training classifiers
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#train-test split with Bag of Words, then apply TF-IDF to have same set of sample data
X_bow_train, X_bow_test, y_train, y_test = train_test_split(X_bow, y, test_size = 0.2)

y_train_arr = y_train.to_numpy().reshape(-1)

X_tfidf_train = tfidf_transformer.fit_transform(X_bow_train)
X_tfidf_train.shape
X_tfidf_test = tfidf_transformer.fit_transform(X_bow_test)
X_tfidf_test.shape

print('Classifier\t Feature Extr.\t Train Acc.\t Test Acc.')
print('---------------------------------------------------------')

#naive bayes
from sklearn.naive_bayes import MultinomialNB
clf_bow = MultinomialNB().fit(X_bow_train, y_train_arr)
clf_tfidf = MultinomialNB().fit(X_tfidf_train, y_train_arr)
print('Naive Bayes\t BoW\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = clf_bow.predict(X_bow_train), y_true = y_train),
                                                              accuracy_score(y_pred = clf_bow.predict(X_bow_test), y_true = y_test)))
print('Naive Bayes\t TF-IDF\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = clf_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                                 accuracy_score(y_pred = clf_tfidf.predict(X_tfidf_test), y_true = y_test)))

print('---------------------------------------------------------')
#support vector machine
from sklearn.linear_model import SGDClassifier
svm_bow = SGDClassifier().fit(X_bow_train, y_train_arr)
svm_tfidf = SGDClassifier().fit(X_tfidf_train, y_train_arr)
print('SVM\t\t BoW\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svm_bow.predict(X_bow_train), y_true = y_train),
                                                          accuracy_score(y_pred = svm_bow.predict(X_bow_test), y_true = y_test)))
print('SVM\t\t TF-IDF\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svm_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                             accuracy_score(y_pred = svm_tfidf.predict(X_tfidf_test), y_true = y_test)))


print('---------------------------------------------------------')
#logistic regression
from sklearn.linear_model import LogisticRegression
lr_bow = LogisticRegression().fit(X_bow_train, y_train_arr)
lr_tfidf = LogisticRegression().fit(X_tfidf_train, y_train_arr)
print('Log Regression\t BoW\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = lr_bow.predict(X_bow_train), y_true = y_train),
                                               accuracy_score(y_pred = lr_bow.predict(X_bow_test), y_true = y_test)))
print('Log Regression\t TF-IDF\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = lr_tfidf.predict(X_tfidf_train), y_true = y_train),
                                               accuracy_score(y_pred = lr_tfidf.predict(X_tfidf_test), y_true = y_test)))

print('---------------------------------------------------------')
#linear support vector classification
from sklearn.svm import LinearSVC
svc_bow = LinearSVC(max_iter = 2000).fit(X_bow_train, y_train_arr)
svc_tfidf = LinearSVC().fit(X_tfidf_train, y_train_arr)
print('SVC\t\t BoW\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svc_bow.predict(X_bow_train), y_true = y_train),
                                               accuracy_score(y_pred = svc_bow.predict(X_bow_test), y_true = y_test)))
print('SVC\t\t TF-IDF\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svc_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                  accuracy_score(y_pred = svc_tfidf.predict(X_tfidf_test), y_true = y_test)))



Classifier	 Feature Extr.	 Train Acc.	 Test Acc.
---------------------------------------------------------
Naive Bayes	 BoW		 0.91494	 0.73800
Naive Bayes	 TF-IDF		 0.91394	 0.74100
---------------------------------------------------------
SVM		 BoW		 0.95969	 0.73850
SVM		 TF-IDF		 0.88538	 0.75350
---------------------------------------------------------
Log Regression	 BoW		 0.93331	 0.74375
Log Regression	 TF-IDF		 0.86675	 0.75300
---------------------------------------------------------
SVC		 BoW		 0.98931	 0.72100
SVC		 TF-IDF		 0.97469	 0.74475


In [11]:
#hyperparameter testing
from sklearn.model_selection import GridSearchCV
print('Testing for hyperparameters:')

#naive bayes
clf_param = {
    'fit_prior': (True, False),
    'force_alpha': (True, False),
    'alpha': (1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1),
}

gs_clf = GridSearchCV(clf_tfidf, clf_param, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(X_tfidf_train, y_train_arr)
print('Naive Bayes:')
for param_name in sorted(clf_param.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))
print('---------------------------------------------------------')



#support vector machine
svm_param = {
    'max_iter': (1500, 1000, 500),
    'loss': ('hinge', 'perceptron', 'squared_error'),
    'alpha': (0.01, 0.005, 0.001),
}

gs_svm = GridSearchCV(svm_tfidf, svm_param, cv=5, n_jobs=-1)
gs_svm = gs_svm.fit(X_tfidf_train, y_train_arr)
print('SVM:')
for param_name in sorted(svm_param.keys()):
    print("%s: %r" % (param_name, gs_svm.best_params_[param_name]))
print('---------------------------------------------------------')



#logistic regression
lr_param = {
    'solver': ('lbfgs', 'liblinear', 'newton-cg'),
    'max_iter': (500, 750, 1000, 1250)
}

gs_lr = GridSearchCV(lr_tfidf, lr_param, cv=5, n_jobs=-1)
gs_lr = gs_lr.fit(X_tfidf_train, y_train_arr)
print('Logistic Regression:')
for param_name in sorted(lr_param.keys()):
    print("%s: %r" % (param_name, gs_lr.best_params_[param_name]))
print('---------------------------------------------------------')



#linear support vector classification
svc_param = {
    'max_iter': (500, 750, 1000, 2000, 2500),
    'C': (0.01, 0.1, 0.5, 1.0, 1.5),
}

gs_svc = GridSearchCV(svc_tfidf, svc_param, cv=5, n_jobs=-1)
gs_svc = gs_svc.fit(X_tfidf_train, y_train_arr)
print('SVM:')
for param_name in sorted(svc_param.keys()):
    print("%s: %r" % (param_name, gs_svc.best_params_[param_name]))
print('---------------------------------------------------------')

Testing for hyperparameters:
Naive Bayes:
alpha: 1.0
fit_prior: True
force_alpha: True
---------------------------------------------------------
SVM:
alpha: 0.001
loss: 'squared_error'
max_iter: 1000
---------------------------------------------------------
Logistic Regression:
max_iter: 500
solver: 'liblinear'
---------------------------------------------------------
SVM:
C: 0.1
max_iter: 500
---------------------------------------------------------


In [12]:
#testing the optimized hyperparameters with TF-IDF
print('Results of applying tuned hyperparameters')
print('-----------------------------------------')

print('Classifier\tTrain Acc.\tTest Acc.')
clf_tfidf = MultinomialNB(alpha = 0.6).fit(X_tfidf_train, y_train_arr)
print('Naive Bayes\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = clf_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                                 accuracy_score(y_pred = clf_tfidf.predict(X_tfidf_test), y_true = y_test)))

svm_tfidf = SGDClassifier(alpha = 0.001, loss = 'squared_error', max_iter = 1500).fit(X_tfidf_train, y_train_arr)
print('SVM\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svm_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                             accuracy_score(y_pred = svm_tfidf.predict(X_tfidf_test), y_true = y_test)))

lr_tfidf = LogisticRegression(solver = 'liblinear', max_iter = 500).fit(X_tfidf_train, y_train_arr)
print('Log Regression\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = lr_tfidf.predict(X_tfidf_train), y_true = y_train),
                                               accuracy_score(y_pred = lr_tfidf.predict(X_tfidf_test), y_true = y_test)))

svc_tfidf = LinearSVC(C = 0.1, max_iter = 500).fit(X_tfidf_train, y_train_arr)
print('SVC\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svc_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                  accuracy_score(y_pred = svc_tfidf.predict(X_tfidf_test), y_true = y_test)))

Results of applying tuned hyperparameters
-----------------------------------------
Classifier	Train Acc.	Test Acc.
Naive Bayes	 0.92994	 0.73275
SVM		 0.81088	 0.74700
Log Regression	 0.86787	 0.75200
SVC		 0.86556	 0.75350


In [13]:
from sklearn.metrics import accuracy_score, classification_report
#WITHOUT TUNING

# Logistic Regression
m_lr = LogisticRegression()
m_lr.fit(X_tfidf_train, y_train)
# Predictions
y_pred_lr_train = m_lr.predict(X_tfidf_train)
y_pred_lr_test = m_lr.predict(X_tfidf_test)

# Accuracy scores
lr_train_accuracy = accuracy_score(y_train, y_pred_lr_train)
lr_test_accuracy = accuracy_score(y_test, y_pred_lr_test)

# Print formatted output
print("Logistic Regression\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(lr_train_accuracy, lr_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_lr_test))



# Linear SVC
m_svc = LinearSVC()
m_svc.fit(X_tfidf_train, y_train)
# Predictions
y_pred_svc_train = m_svc.predict(X_tfidf_train)
y_pred_svc_test = m_svc.predict(X_tfidf_test)

# Accuracy scores
svc_train_accuracy = accuracy_score(y_train, y_pred_svc_train)
svc_test_accuracy = accuracy_score(y_test, y_pred_svc_test)

# Print formatted output
print("Linear SVC\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(svc_train_accuracy, svc_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_svc_test))



# Support vector machine
m_svm = SGDClassifier()
m_svm.fit(X_tfidf_train, y_train)
# Predictions
y_pred_svm_train = m_svm.predict(X_tfidf_train)
y_pred_svm_test = m_svm.predict(X_tfidf_test)

# Accuracy scores
svm_train_accuracy = accuracy_score(y_train, y_pred_svm_train)
svm_test_accuracy = accuracy_score(y_test, y_pred_svm_test)

# Print formatted output
print("Support Vector Machine\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(svm_train_accuracy, svm_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_svm_test))



# naive bayes
m_clf = MultinomialNB()
m_clf.fit(X_tfidf_train, y_train)
# Predictions
y_pred_clf_train = m_clf.predict(X_tfidf_train)
y_pred_clf_test = m_clf.predict(X_tfidf_test)

# Accuracy scores
clf_train_accuracy = accuracy_score(y_train, y_pred_clf_train)
clf_test_accuracy = accuracy_score(y_test, y_pred_clf_test)

# Print formatted output
print("Naive Bayes\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(clf_train_accuracy, clf_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_clf_test))

Logistic Regression	 Train Accuracy: 0.86675	 Test Accuracy: 0.75300
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.74      0.75      2021
           1       0.74      0.77      0.76      1979

    accuracy                           0.75      4000
   macro avg       0.75      0.75      0.75      4000
weighted avg       0.75      0.75      0.75      4000

Linear SVC	 Train Accuracy: 0.97469	 Test Accuracy: 0.74475
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.74      0.75      2021
           1       0.74      0.75      0.74      1979

    accuracy                           0.74      4000
   macro avg       0.74      0.74      0.74      4000
weighted avg       0.74      0.74      0.74      4000

Support Vector Machine	 Train Accuracy: 0.88581	 Test Accuracy: 0.75250
Classification Report:
               precision    recall  f1-score   support

           0      

In [14]:
from sklearn.metrics import accuracy_score, classification_report
#WITH TUNING


# Logistic Regression
# Predictions
y_pred_lr_train = gs_lr.predict(X_tfidf_train)
y_pred_lr_test = gs_lr.predict(X_tfidf_test)

# Accuracy scores
lr_train_accuracy = accuracy_score(y_train, y_pred_lr_train)
lr_test_accuracy = accuracy_score(y_test, y_pred_lr_test)

print ("Logistic Regression")
for param_name in sorted(lr_param.keys()):
    print("%s: %r" % (param_name, gs_lr.best_params_[param_name]))
print('---------------------------------------------------------')

# Print formatted output
print("Logistic Regression\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(lr_train_accuracy, lr_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_lr_test))



# Linear SVC
# Predictions
y_pred_svc_train = gs_svc.predict(X_tfidf_train)
y_pred_svc_test = gs_svc.predict(X_tfidf_test)

# Accuracy scores
svc_train_accuracy = accuracy_score(y_train, y_pred_svc_train)
svc_test_accuracy = accuracy_score(y_test, y_pred_svc_test)

print ("Linear SVC")
for param_name in sorted(svc_param.keys()):
    print("%s: %r" % (param_name, gs_svc.best_params_[param_name]))
print('---------------------------------------------------------')

# Print formatted output
print("Linear SVC\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(svc_train_accuracy, svc_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_svc_test))



# Support vector machine
# Predictions
y_pred_svm_train = gs_svm.predict(X_tfidf_train)
y_pred_svm_test = gs_svm.predict(X_tfidf_test)

# Accuracy scores
svm_train_accuracy = accuracy_score(y_train, y_pred_svm_train)
svm_test_accuracy = accuracy_score(y_test, y_pred_svm_test)

print("Support Vector Machine")
for param_name in sorted(svm_param.keys()):
    print("%s: %r" % (param_name, gs_svm.best_params_[param_name]))
print('---------------------------------------------------------')

# Print formatted output
print("Support Vector Machine\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(svm_train_accuracy, svm_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_svm_test))



# naive bayes
# Predictions
y_pred_clf_train = gs_clf.predict(X_tfidf_train)
y_pred_clf_test = gs_clf.predict(X_tfidf_test)

# Accuracy scores
clf_train_accuracy = accuracy_score(y_train, y_pred_clf_train)
clf_test_accuracy = accuracy_score(y_test, y_pred_clf_test)

print("Naive Bayes")
for param_name in sorted(clf_param.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))
print('---------------------------------------------------------')

# Print formatted output
print("Naive Bayes\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(clf_train_accuracy, clf_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_clf_test))

Logistic Regression
max_iter: 500
solver: 'liblinear'
---------------------------------------------------------
Logistic Regression	 Train Accuracy: 0.86787	 Test Accuracy: 0.75200
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.74      0.75      2021
           1       0.74      0.77      0.75      1979

    accuracy                           0.75      4000
   macro avg       0.75      0.75      0.75      4000
weighted avg       0.75      0.75      0.75      4000

Linear SVC
C: 0.1
max_iter: 500
---------------------------------------------------------
Linear SVC	 Train Accuracy: 0.86556	 Test Accuracy: 0.75350
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.73      0.75      2021
           1       0.74      0.78      0.76      1979

    accuracy                           0.75      4000
   macro avg       0.75      0.75      0.75      4000
weighted avg       0.7